<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h1><font color="#00586D" size=5>Trabajo Fin de Master (TFM)</font></h1>
<h2><font color="#00586D" size=5>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font color="#00586D" size=4>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<br>
<h1><font color="#00586D" size=5>Alumna: Luna Jiménez Fernández</font></h1>
<br><br><br>


<div align="right">
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>
<br>

---
<br>

INTRO A LA LIBRETA

INDICE

TITULOS ASI:

<a id="section1"></a>
## <font color="#00586D"> 1. Inspección y exploración inicial </font>
<br>